# People Analytics Case Study

## 1. Libraries

In [2]:
# import libraries
import pandas as pd
import pymysql
from sqlalchemy import create_engine
import getpass

## 2. Data Overview

In [3]:
# load data
df = pd.read_csv('../data/raw/hr_data.csv')

In [4]:
# View the DataFrame
df

,id,first_name,last_name,birthdate,gender,race,department,jobtitle,location,hire_date,termdate,location_city,location_state
0,00-0037846,Kimmy,Walczynski,6/4/1991,Male,Hispanic or Latino,Engineering,Programmer Analyst I,Headquarters,1/20/2002,NaN,Cleveland,Ohio
1,00-0041533,Ignatius,Springett,6/29/1984,Male,White,Business Development,Business Analyst,Headquarters,4/8/2019,NaN,Cleveland,Ohio
2,00-0045747,Corbie,Bittlestone,7/29/1989,Male,Black or African American,Sales,Solutions Engineer Manager,Headquarters,10/12/2010,NaN,Cleveland,Ohio
3,00-0055274,Baxy,Matton,9/14/1982,Female,White,Services,Service Tech,Headquarters,4/10/2005,NaN,Cleveland,Ohio
4,00-0076100,Terrell,Suff,4/11/1994,Female,Two or More Races,Product Management,Business Analyst,Remote,9/29/2010,2029-10-29 06:09:38 UTC,Flint,Michigan
...,...,...,...,...,...,...,...,...,...,...,...,...,...
37403,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37404,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37405,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37406,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# General information about the DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37408 entries, 0 to 37407
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              22214 non-null  object
 1   first_name      22214 non-null  object
 2   last_name       22214 non-null  object
 3   birthdate       22214 non-null  object
 4   gender          22214 non-null  object
 5   race            22214 non-null  object
 6   department      22214 non-null  object
 7   jobtitle        22267 non-null  object
 8   location        22214 non-null  object
 9   hire_date       22214 non-null  object
 10  termdate        3929 non-null   object
 11  location_city   22214 non-null  object
 12  location_state  22214 non-null  object
dtypes: object(13)
memory usage: 3.7+ MB


## 3. Data Cleaning

In [6]:
# Rename columns in the DataFrame
df = df.rename(columns={
    'birthdate': 'birth_date',
    'jobtitle': 'job_title',
    'termdate': 'term_date',
    'id': 'emp_id'
})

In [7]:
# Remove rows where all values are NaN (empty)
df = df.dropna(how='all')

In [8]:
df

,emp_id,first_name,last_name,birth_date,gender,race,department,job_title,location,hire_date,term_date,location_city,location_state
0,00-0037846,Kimmy,Walczynski,6/4/1991,Male,Hispanic or Latino,Engineering,Programmer Analyst I,Headquarters,1/20/2002,NaN,Cleveland,Ohio
1,00-0041533,Ignatius,Springett,6/29/1984,Male,White,Business Development,Business Analyst,Headquarters,4/8/2019,NaN,Cleveland,Ohio
2,00-0045747,Corbie,Bittlestone,7/29/1989,Male,Black or African American,Sales,Solutions Engineer Manager,Headquarters,10/12/2010,NaN,Cleveland,Ohio
3,00-0055274,Baxy,Matton,9/14/1982,Female,White,Services,Service Tech,Headquarters,4/10/2005,NaN,Cleveland,Ohio
4,00-0076100,Terrell,Suff,4/11/1994,Female,Two or More Races,Product Management,Business Analyst,Remote,9/29/2010,2029-10-29 06:09:38 UTC,Flint,Michigan
...,...,...,...,...,...,...,...,...,...,...,...,...,...
22433,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Support Staff III,NaN,NaN,NaN,NaN,NaN
22434,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Support Staff III,NaN,NaN,NaN,NaN,NaN
22435,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Support Staff III,NaN,NaN,NaN,NaN,NaN
22436,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Support Staff III,NaN,NaN,NaN,NaN,NaN


In [9]:
# Check for duplicated rows
df.duplicated().sum()

50

In [10]:
# Show duplicated rows
df[df.duplicated()]

,emp_id,first_name,last_name,birth_date,gender,race,department,job_title,location,hire_date,term_date,location_city,location_state
22322,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Support Staff II,NaN,NaN,NaN,NaN,NaN
22323,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Support Staff II,NaN,NaN,NaN,NaN,NaN
22324,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Support Staff II,NaN,NaN,NaN,NaN,NaN
22325,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Support Staff II,NaN,NaN,NaN,NaN,NaN
22326,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Support Staff II,NaN,NaN,NaN,NaN,NaN
22327,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Support Staff II,NaN,NaN,NaN,NaN,NaN
22328,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Support Staff II,NaN,NaN,NaN,NaN,NaN
22329,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Support Staff II,NaN,NaN,NaN,NaN,NaN
22330,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Support Staff II,NaN,NaN,NaN,NaN,NaN
22331,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Support Staff II,NaN,NaN,NaN,NaN,NaN


In [11]:
# Remove rows where all columns except 'jobtitle' are NaN
df = df.dropna(how='all', subset=df.columns.difference(['job_title']))

In [12]:
# Count the occurrences of each value in the 'id' column
duplicate_counts = df['emp_id'].value_counts()

# Filter the counts to show only duplicates
duplicates = duplicate_counts[duplicate_counts > 1]

# Show the duplicated ids and their counts
print(duplicates)

Series([], Name: count, dtype: int64)


In [13]:
# Dropping the 'location_city' column as it's not relevant for the analysis
df.drop(columns=['location_city'], inplace=True)

/var/folders/lb/xlcp_nw57bz0k682c287nrk00000gn/T/ipykernel_70839/1230876464.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['location_city'], inplace=True)


In [14]:
# Count unique values in 'location_state'
df['location_state'].value_counts()

location_state
Ohio            18025
Pennsylvania     1115
Illinois          868
Indiana           700
Michigan          673
Kentucky          451
Wisconsin         382
Name: count, dtype: int64

In [15]:
# Mapping states to California and neighboring states for storytelling purposes.
# The original dataset is fictitious, allowing for creative flexibility in the analysis.

state_mapping = {
    'Ohio': 'California',
    'Pennsylvania': 'Oregon',
    'Illinois': 'Nevada',
    'Indiana': 'Arizona',
    'Michigan': 'Utah',
    'Kentucky': 'New Mexico',
    'Wisconsin': 'Idaho'
}

# Apply the mapping to the 'location_state' column
df['location_state'] = df['location_state'].replace(state_mapping)


/var/folders/lb/xlcp_nw57bz0k682c287nrk00000gn/T/ipykernel_70839/2314722448.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['location_state'] = df['location_state'].replace(state_mapping)


In [16]:
# Count unique values in 'location_state'
df['location_state'].value_counts()

location_state
California    18025
Oregon         1115
Nevada          868
Arizona         700
Utah            673
New Mexico      451
Idaho           382
Name: count, dtype: int64

In [17]:
df.dtypes

emp_id            object
first_name        object
last_name         object
birth_date        object
gender            object
race              object
department        object
job_title         object
location          object
hire_date         object
term_date         object
location_state    object
dtype: object

In [18]:
# Count the number of valid (non-null) entries in 'birth_date', 'hire_date', and 'term_date'
valid_counts = df[['birth_date', 'hire_date', 'term_date']].count()

# Print the counts of valid entries for each column
print(valid_counts)

birth_date    22214
hire_date     22214
term_date      3929
dtype: int64


In [19]:
df

,emp_id,first_name,last_name,birth_date,gender,race,department,job_title,location,hire_date,term_date,location_state
0,00-0037846,Kimmy,Walczynski,6/4/1991,Male,Hispanic or Latino,Engineering,Programmer Analyst I,Headquarters,1/20/2002,NaN,California
1,00-0041533,Ignatius,Springett,6/29/1984,Male,White,Business Development,Business Analyst,Headquarters,4/8/2019,NaN,California
2,00-0045747,Corbie,Bittlestone,7/29/1989,Male,Black or African American,Sales,Solutions Engineer Manager,Headquarters,10/12/2010,NaN,California
3,00-0055274,Baxy,Matton,9/14/1982,Female,White,Services,Service Tech,Headquarters,4/10/2005,NaN,California
4,00-0076100,Terrell,Suff,4/11/1994,Female,Two or More Races,Product Management,Business Analyst,Remote,9/29/2010,2029-10-29 06:09:38 UTC,Utah
...,...,...,...,...,...,...,...,...,...,...,...,...
22209,99-9797418,Dorella,Garvan,7/8/1998,Female,Hispanic or Latino,Research and Development,Research Assistant I,Headquarters,2/8/2012,NaN,California
22210,99-9869877,Dasie,Thorsby,4/19/2001,Female,Two or More Races,Services,Service Manager,Headquarters,10/6/2017,NaN,California
22211,99-9919822,Nerty,Wilding,2/9/1970,Female,Two or More Races,Training,Junior Trainer,Headquarters,2/8/2001,NaN,California
22212,99-9960380,Mabelle,Dawks,9/2/1985,Male,Two or More Races,Accounting,Staff Accountant I,Headquarters,4/3/2005,2012-12-10 14:29:59 UTC,California


In [20]:
# Convert 'birth_date' and 'hire_date' to datetime format
df['birth_date'] = pd.to_datetime(df['birth_date'], errors='coerce')
df['hire_date'] = pd.to_datetime(df['hire_date'], errors='coerce')

/var/folders/lb/xlcp_nw57bz0k682c287nrk00000gn/T/ipykernel_70839/1115299221.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['birth_date'] = pd.to_datetime(df['birth_date'], errors='coerce')
/var/folders/lb/xlcp_nw57bz0k682c287nrk00000gn/T/ipykernel_70839/1115299221.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['hire_date'] = pd.to_datetime(df['hire_date'], errors='coerce')


In [21]:
# Convert the 'term_date' column from string to datetime format and extract only the date
df['term_date'] = pd.to_datetime(df['term_date']).dt.date

# Convert the 'term_date' column back to datetime format with default time (00:00:00)
df['term_date'] = pd.to_datetime(df['term_date'])

/var/folders/lb/xlcp_nw57bz0k682c287nrk00000gn/T/ipykernel_70839/1376337505.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['term_date'] = pd.to_datetime(df['term_date']).dt.date
/var/folders/lb/xlcp_nw57bz0k682c287nrk00000gn/T/ipykernel_70839/1376337505.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['term_date'] = pd.to_datetime(df['term_date'])


In [22]:
# Count the number of valid (non-null) entries in 'birth_date', 'hire_date', and 'term_date'
valid_counts = df[['birth_date', 'hire_date', 'term_date']].count()

# Print the counts of valid entries for each column
print(valid_counts)

birth_date    22214
hire_date     22214
term_date      3929
dtype: int64


In [23]:
# General information about the DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22214 entries, 0 to 22213
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   emp_id          22214 non-null  object        
 1   first_name      22214 non-null  object        
 2   last_name       22214 non-null  object        
 3   birth_date      22214 non-null  datetime64[ns]
 4   gender          22214 non-null  object        
 5   race            22214 non-null  object        
 6   department      22214 non-null  object        
 7   job_title       22214 non-null  object        
 8   location        22214 non-null  object        
 9   hire_date       22214 non-null  datetime64[ns]
 10  term_date       3929 non-null   datetime64[ns]
 11  location_state  22214 non-null  object        
dtypes: datetime64[ns](3), object(9)
memory usage: 2.2+ MB


In [24]:
# Create a dictionary to store the min and max values for clarity
date_ranges = {
    'Birth Date': {'Min': df['birth_date'].min(), 'Max': df['birth_date'].max()},
    'Hire Date': {'Min': df['hire_date'].min(), 'Max': df['hire_date'].max()},
    'Term Date': {'Min': df['term_date'].min(), 'Max': df['term_date'].max()}
}

# Convert to a DataFrame for a cleaner output
date_ranges_df = pd.DataFrame(date_ranges)

# Show the date ranges
date_ranges_df

,Birth Date,Hire Date,Term Date
Min,1965-10-16,2000-10-17,2001-04-15
Max,2002-09-13,2020-12-13,2041-10-29


In [25]:
# Get the current timestamp
current_timestamp = pd.Timestamp('now')

# Filter rows where 'term_date' is greater than current date
term_date_invalid = df[df['term_date'] > current_timestamp]

# Show the filtered rows
term_date_invalid

,emp_id,first_name,last_name,birth_date,gender,race,department,job_title,location,hire_date,term_date,location_state
4,00-0076100,Terrell,Suff,1994-04-11,Female,Two or More Races,Product Management,Business Analyst,Remote,2010-09-29,2029-10-29,Utah
27,00-1268049,Fay,Monnelly,1966-07-09,Male,Native Hawaiian or Other Pacific Islander,Engineering,Software Engineer I,Headquarters,2010-02-24,2030-03-21,California
40,00-1792130,Nobe,Leathe,1993-07-23,Male,Black or African American,Engineering,Developer III,Headquarters,2011-01-23,2024-12-07,California
57,00-2623755,Chrysa,Brownell,1983-04-25,Male,White,Engineering,Administrative Officer,Headquarters,2018-02-22,2027-02-01,California
139,00-6479395,Aura,Steagall,1978-07-19,Male,White,Accounting,Staff Accountant I,Headquarters,2013-03-28,2030-02-23,California
...,...,...,...,...,...,...,...,...,...,...,...,...
22038,99-1005402,Cornela,Livermore,1969-10-14,Female,Two or More Races,Engineering,Software Test Engineer I,Remote,2013-06-02,2030-01-30,Nevada
22048,99-1707394,Patrick,Musicka,1976-04-23,Female,Two or More Races,Human Resources,Senior Recruiter,Headquarters,2016-11-25,2026-08-01,California
22083,99-3706255,Nappy,Burchess,1999-10-22,Male,Native Hawaiian or Other Pacific Islander,Accounting,Budget/Accounting Analyst II,Headquarters,2017-12-31,2035-08-20,California
22095,99-4396036,Flory,Hardy-Piggin,1989-03-28,Male,Black or African American,Accounting,Administrative Officer,Remote,2019-03-26,2027-03-03,Arizona


In [26]:
# Drop rows where 'term_date' is greater than the current date
df = df.drop(df[df['term_date'] > current_timestamp].index)

# Check the number of rows after the removal
remaining_rows = len(df)
remaining_rows

20884

In [27]:
# Get distinct values in the 'gender' column
distinct_gender = df['gender'].unique()

# Show the unique values
distinct_gender

array(['Male', 'Female', 'Non-Conforming'], dtype=object)

In [28]:
# Get distinct values in the 'race' column
distinct_race = df['race'].unique()

# Show the unique values
distinct_race

array(['Hispanic or Latino', 'White', 'Black or African American',
       'Asian', 'Two or More Races', 'American Indian or Alaska Native',
       'Native Hawaiian or Other Pacific Islander'], dtype=object)

In [29]:
# Check for empty values in 'race' and 'gender' columns
empty_values = df[df['race'].isnull() | df['gender'].isnull()]

# Show the rows with empty values
empty_values

,emp_id,first_name,last_name,birth_date,gender,race,department,job_title,location,hire_date,term_date,location_state


In [30]:
# Check for null values in all columns
null_counts = df.isnull().sum()

# Show columns with null values
null_counts[null_counts > 0]

term_date    18285
dtype: int64

In [31]:
# Calculate age based on 'birth_date'
current_year = pd.Timestamp('now').year
age = current_year - df['birth_date'].dt.year

# Insert the 'age' column at index 4 (fifth position)
df.insert(4, 'age', age)

# Display the DataFrame with the new 'age' column
df

,emp_id,first_name,last_name,birth_date,age,gender,race,department,job_title,location,hire_date,term_date,location_state
0,00-0037846,Kimmy,Walczynski,1991-06-04,33,Male,Hispanic or Latino,Engineering,Programmer Analyst I,Headquarters,2002-01-20,NaT,California
1,00-0041533,Ignatius,Springett,1984-06-29,40,Male,White,Business Development,Business Analyst,Headquarters,2019-04-08,NaT,California
2,00-0045747,Corbie,Bittlestone,1989-07-29,35,Male,Black or African American,Sales,Solutions Engineer Manager,Headquarters,2010-10-12,NaT,California
3,00-0055274,Baxy,Matton,1982-09-14,42,Female,White,Services,Service Tech,Headquarters,2005-04-10,NaT,California
5,00-0116166,Kacie,Offiler,1971-01-18,53,Male,Asian,Engineering,Developer III,Headquarters,2018-09-01,NaT,California
...,...,...,...,...,...,...,...,...,...,...,...,...,...
22209,99-9797418,Dorella,Garvan,1998-07-08,26,Female,Hispanic or Latino,Research and Development,Research Assistant I,Headquarters,2012-02-08,NaT,California
22210,99-9869877,Dasie,Thorsby,2001-04-19,23,Female,Two or More Races,Services,Service Manager,Headquarters,2017-10-06,NaT,California
22211,99-9919822,Nerty,Wilding,1970-02-09,54,Female,Two or More Races,Training,Junior Trainer,Headquarters,2001-02-08,NaT,California
22212,99-9960380,Mabelle,Dawks,1985-09-02,39,Male,Two or More Races,Accounting,Staff Accountant I,Headquarters,2005-04-03,2012-12-10,California


In [32]:
# Calculate min, avg, and max for the 'age' column
min_age = df['age'].min()
avg_age = df['age'].mean()
max_age = df['age'].max()

# Display the results
age_statistics = {
    'min_age': min_age,
    'avg_age': avg_age,
    'max_age': max_age
}

# Count the number of rows where age is less than 18
count_under_18 = df[df['age'] < 18].shape[0]

# Display the count
age_statistics, count_under_18


({'min_age': 22, 'avg_age': 40.19550852327141, 'max_age': 59}, 0)

In [33]:
df.dtypes

emp_id                    object
first_name                object
last_name                 object
birth_date        datetime64[ns]
age                        int32
gender                    object
race                      object
department                object
job_title                 object
location                  object
hire_date         datetime64[ns]
term_date         datetime64[ns]
location_state            object
dtype: object

In [37]:
# Calcular el total de empleados y las terminaciones por job_title
job_title_count = df.groupby('job_title').agg(
    total_count=('job_title', 'size'),
    termination_count=('term_date', lambda x: x.notnull().sum())
).reset_index()

# Calcular la tasa de rotación
job_title_count['turnover_rate'] = (job_title_count['termination_count'] / job_title_count['total_count']) * 100

# Ordenar los resultados por tasa de rotación en orden descendente
results = job_title_count.sort_values(by='turnover_rate', ascending=False)

# Mostrar los resultados
results.head(50)

,job_title,total_count,termination_count,turnover_rate
156,Statistician IV,1,1,100.000000
68,Executive Secretary,1,1,100.000000
94,Office Assistant II,1,1,100.000000
155,Statistician III,4,2,50.000000
122,Sales Representative,5,2,40.000000
64,Engineer II,6,2,33.333333
98,Payment Adjustment Coordinator,23,7,30.434783
121,Sales Associate,7,2,28.571429
78,Human Resources Assistant I,14,4,28.571429
71,General Manager,29,8,27.586207


In [38]:
# Calculate the turnover rate for each job title
turnover_rate = df.groupby('job_title').agg(
    total_count=('job_title', 'size'),
    termination_count=('term_date', lambda x: x.notnull().sum())
).reset_index()

# Calculate the turnover rate as a percentage
turnover_rate['turnover_rate'] = (turnover_rate['termination_count'] / turnover_rate['total_count']) * 100

# Sort the results by total_count in descending order
turnover_rate_sorted = turnover_rate.sort_values(by='total_count', ascending=False)

# Display the sorted results
turnover_rate_sorted

,job_title,total_count,termination_count,turnover_rate
117,Research Assistant II,705,71,10.070922
33,Business Analyst,660,86,13.030303
77,Human Resources Analyst II,570,75,13.157895
116,Research Assistant I,501,67,13.373253
1,Account Executive,469,60,12.793177
...,...,...,...,...
23,Associate Professor,1,0,0.000000
95,Office Assistant IV,1,0,0.000000
68,Executive Secretary,1,1,100.000000
156,Statistician IV,1,1,100.000000


KeyError: 'total_count'

## 4. Exporting the DataFrame to MySQL Workbench

In [35]:
# Connection parameters
bd = "hr_data"
password = "password"
connection_string = 'mysql+pymysql://root:' + password + '@localhost/' + bd

# Create the connection engine
engine = create_engine(connection_string)

# Send the DataFrame named 'df' to MySQL
table_name = 'employees'
df.to_sql(table_name, con=engine, if_exists='replace', index=False)

20884